# **Data Science Machine Learning Project 2**

For this Project I was tasked to find my own dataset that not only fit certain criteria, but also something that interests me or my future endeavors.

I chose a dataset that pertains to the healthcare industry. As I come from a pharmacy background, and would like to work in data science in the healthcare industry this dataset uncompases both these goals.

**Context**

According to the World Health Organization (WHO) stroke is the 2nd leading cause of death globally, responsible for approximately 11% of total deaths.
This dataset is used to predict whether a patient is likely to get stroke based on the input parameters like gender, age, various diseases, and smoking status. Each row in the data provides relavant information about the patient.

# Part 1 Choosing a Dataset

**Project 2 - Part 1** 

For this task:

Create a Colab notebook where you have uploaded and shown the .head() of each of your data sets.  For each of the proposed datasets, answer the following questions:

1) Source of data

Kaggle: https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset?resource=download

2) Brief description of data

This dataset is used to predict whether a patient is likely to get stroke based on the input parameters like gender, age, various diseases, and smoking status. Each row in the data provides relavant information about the patient.

3) What is the target?

If a patient will have a stroke or not

4) Is this a classification or regression problem?

Classification

5) How many features?

12

6) How many rows of data.

583

7) What, if any, challenges do your foresee in cleaning, exploring, or modeling with this dataset?

Being able to tune the data to be accurate and precicise enough to be able to evaluate if a patient will have a stroke. Being able to explore and explain the data in a way that makes sense and leads to logical conclusions.

# Mount Drive / Import Libs / Load Dataset / Check Info

In [6]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.tree import plot_tree, DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, plot_confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn import set_config
set_config(display='diagram')

In [8]:
# Load dataset 
df= '/content/healthcare-dataset-stroke-data.csv'
df= pd.read_csv(df, index_col=0)
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,,
9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [9]:
# Get a quick overview of the data within the columns and what data type the column provides
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5110 entries, 9046 to 44679
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             5110 non-null   object 
 1   age                5110 non-null   float64
 2   hypertension       5110 non-null   int64  
 3   heart_disease      5110 non-null   int64  
 4   ever_married       5110 non-null   object 
 5   work_type          5110 non-null   object 
 6   Residence_type     5110 non-null   object 
 7   avg_glucose_level  5110 non-null   float64
 8   bmi                4909 non-null   float64
 9   smoking_status     5110 non-null   object 
 10  stroke             5110 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 479.1+ KB


# Part 2 Clean the Data

**Project 2 - Part 2** 

Using the approved dataset you proposed for project 2 part 1 last week, you will clean and explore the data set.  

To meet the requirements for this week you must have:

-deleted unnecessary columns

-deleted duplicate rows

-identified and addressed missing values 

-identified and corrected inconsistencies in data for categorical values (i.e. Cat, cat, cats)

-produced univariate visuals for the target and all features

-identified outliers

-clearly commented all of your cleaning steps and described any decisions you made  


# Data Cleaning

In [10]:
# Check for Duplicates
df.duplicated().sum()

0

In [11]:
# Identify and address any missing values in this dataset. 
df.isna().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

Since 40% of the data for the BMI column is missing I will choose to drop this column.

In [24]:
df = df.drop(columns=['bmi'])
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,smoking_status,stroke
id,,,,,,,,,,
9046,Male,67.0,0,1,Yes,Private,Urban,228.69,formerly smoked,1
51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,never smoked,1
31112,Male,80.0,0,1,Yes,Private,Rural,105.92,never smoked,1
60182,Female,49.0,0,0,Yes,Private,Urban,171.23,smokes,1
1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,never smoked,1


In [12]:
# Check unique values in dataset
df.apply(lambda x: len(x.unique()))

gender                  3
age                   104
hypertension            2
heart_disease           2
ever_married            2
work_type               5
Residence_type          2
avg_glucose_level    3979
bmi                   419
smoking_status          4
stroke                  2
dtype: int64

In [13]:
# Identify and address any inappropriate or unusual numberical data points (example: age cannot be 325)
data_types = df.dtypes
numerical_data_types = data_types[(data_types == "float64") | (data_types == "int64")]
numerical_data_types

age                  float64
hypertension           int64
heart_disease          int64
avg_glucose_level    float64
bmi                  float64
stroke                 int64
dtype: object

In [14]:
# Check for Outliers
df.describe()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,5110.000000,5110.000000,5110.000000,5110.000000,4909.000000,5110.000000
mean,43.226614,0.097456,0.054012,106.147677,28.893237,0.048728
std,22.612647,0.296607,0.226063,45.283560,7.854067,0.215320
min,0.080000,0.000000,0.000000,55.120000,10.300000,0.000000
25%,25.000000,0.000000,0.000000,77.245000,23.500000,0.000000
50%,45.000000,0.000000,0.000000,91.885000,28.100000,0.000000
75%,61.000000,0.000000,0.000000,114.090000,33.100000,0.000000
max,82.000000,1.000000,1.000000,271.740000,97.600000,1.000000


In [15]:
# Identify and address any inconsistencies in categorical values
# Double check for anymore random values, mispellings or duplicates
dtypes = df.dtypes
str_cols = dtypes[dtypes=='object'].index
str_cols

Index(['gender', 'ever_married', 'work_type', 'Residence_type',
       'smoking_status'],
      dtype='object')

In [16]:
for col in str_cols:
    print(f"- Column= {col}")
    print(df[col].value_counts(dropna=False))
    print('\n\n')

- Column= gender
Female    2994
Male      2115
Other        1
Name: gender, dtype: int64



- Column= ever_married
Yes    3353
No     1757
Name: ever_married, dtype: int64



- Column= work_type
Private          2925
Self-employed     819
children          687
Govt_job          657
Never_worked       22
Name: work_type, dtype: int64



- Column= Residence_type
Urban    2596
Rural    2514
Name: Residence_type, dtype: int64



- Column= smoking_status
never smoked       1892
Unknown            1544
formerly smoked     885
smokes              789
Name: smoking_status, dtype: int64





# Visual the Target and Features

In [17]:
!pip install autoviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
from autoviz.AutoViz_Class import AutoViz_Class
AV= AutoViz_Class()
sep = ","
df_av= AV.AutoViz('/content/healthcare-dataset-stroke-data.csv',
                  sep= ',',
                  depVar= '',
                  header = 0, 
                  verbose = 1, 
                  lowess = False, 
                  chart_format ='png',
                  max_rows_analyzed = 15000, 
                  max_cols_analyzed = 30)

plt.show()

Shape of your Data Set loaded: (5110, 12)
#######################################################################################
######################## C L A S S I F Y I N G  V A R I A B L E S  ####################
#######################################################################################
Classifying variables in data set...
    Number of Numeric Columns =  3
    Number of Integer-Categorical Columns =  0
    Number of String-Categorical Columns =  3
    Number of Factor-Categorical Columns =  0
    Number of String-Boolean Columns =  2
    Number of Numeric-Boolean Columns =  3
    Number of Discrete String Columns =  0
    Number of NLP String Columns =  0
    Number of Date Time Columns =  0
    Number of ID Columns =  1
    Number of Columns to Delete =  0
    12 Predictors classified...
        1 variables removed since they were ID or low-information variables
        List of variables removed: ['id']
Data Set Shape: 5110 rows, 12 cols
Data Set columns info:
* id: